# Check Both Methods of Face Area Calculation

This notebook investigates the two methods identified for calculating the face area. The first method is to use the optional HEC-RAS HDF5 output "Face Water Surface" with the HEC-RAS geometric preprocessed area-elevation lookup tables. The second method is to also use the optional HEC-RAS HDF5 outputs "Face Velocity" and "Face Flow".

In [1]:
import sys
sys.path.insert(1, '../../src/clearwater_riverine')
import ras2dwq as cwr
import h5py
import xarray
import pandas as pd

## Gather needed inputs from the HEC-RAS HDF5 output file

The HEC-RAS tutorial case Muncie was used for this example. The standard set of HEC-RAS output variables are not sufficient to calculate the face area. HEC-RAS can generate the needed output variable during the hydraulic simulation, but the additional variables need to be selected before the simulation is executed. To turn on the selected output variables open the simulations "Unsteady Flow Analysis" window; under the "Options" menu click on the "Output Options..."; select the "HDF5 Write Parameters" in the "HEC-RAS - Set Output Control Options" window; within the combo box titled "Additional 2D Variable Written to Results File (not used/needed for RAS) select "Face Water Surface", "Face Velocity", and "Face Flow". These additional variables have been selected for the "Unsteady Run with 2D 50ft User n Value R" (Short ID = "50ft User n Regions"; Plan File = "Muncie.p04"). The HDF5 file from this simulation with the additional variables has been renamed to "Muncie.p04_someOptionalOutput.hdf" and a copy has been placed in this respository. Please note if all additional variables were selected the HDF5 file would have exceeded github's 100MB limit. The following code gathers the needed inputs from the HEC-RAS HDF5 output file for both face area calculation methods.

In [2]:
fpath = '../data/Muncie.p04_someOptionalOutput.hdf'

with h5py.File(fpath, 'r') as infile:
    twoDarea_name = cwr.parse_project_name(infile)
    faces_area_elevation_info_df = cwr.hdf_to_pandas(infile[f'Geometry/2D Flow Areas/{twoDarea_name}/Faces Area Elevation Info'])
    faces_area_elevation_values_df = cwr.hdf_to_pandas(infile[f'Geometry/2D Flow Areas/{twoDarea_name}/Faces Area Elevation Values'])
    faces_normalunitvector_and_length_df = cwr.hdf_to_pandas(infile[f'Geometry/2D Flow Areas/{twoDarea_name}/Faces NormalUnitVector and Length'])
    faces_cell_indexes_df = cwr.hdf_to_pandas(infile[f'Geometry/2D Flow Areas/{twoDarea_name}/Faces Cell Indexes'])
    face_water_surface_elev_xr = cwr.hdf_to_xarray(infile[f'Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/2D Flow Areas/{twoDarea_name}/Face Water Surface'], ('time', 'face'))
    face_velocity_xr = cwr.hdf_to_xarray(infile[f'Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/2D Flow Areas/{twoDarea_name}/Face Velocity'], ('time', 'face'))
    face_flow_xr = cwr.hdf_to_xarray(infile[f'Results/Unsteady/Output/Output Blocks/Base Output/Unsteady Time Series/2D Flow Areas/{twoDarea_name}/Face Flow'], ('time', 'face'))

## Find Face Areas from the Face Water Surface Elevation

The new "compute_face_areas_from_faceWSE" function was derived from the "compute_face_areas" function. While troubleshooting the new "compute_face_areas_from_faceWSE" function a bug was discovered in the line "if elev[i] < water_surface_elev:". The fix was this: "if elev[i] < water_surface_elev and elev[i+1] >= water_surface_elev:". This bug is still present in the "compute_face_areas" function, but this function is now obsolete since it produces different face areas depending on which adjacent cell's water surface elevation is being used if a hydraulic gradient exist between these cells for a given time step.


In [3]:
ntimes, nfaces = face_water_surface_elev_xr.shape

face_areas = cwr.compute_face_areas_from_faceWSE(
    face_water_surface_elev_xr.values,
    faces_normalunitvector_and_length_df['Face Length'].values,
    faces_cell_indexes_df['Cell 0'].values,
    faces_area_elevation_info_df['Starting Index'].values,
    faces_area_elevation_info_df['Count'].values,
    faces_area_elevation_values_df['Z'].values,
    faces_area_elevation_values_df['Area'].values,
    ntimes,
    nfaces)
face_areas_df = pd.DataFrame(face_areas, columns = range(nfaces))

print(face_areas_df)   


          0           1           2           3           4           5      \
0      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
1      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
2      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
3      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
4      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
..          ...         ...         ...         ...         ...         ...   
284  223.904129  248.920273  193.206421  272.562653  293.375183  239.892853   
285  223.843002  248.849930  193.149933  272.492462  293.301941  239.824234   
286  223.779205  248.776535  193.090988  272.419220  293.228699  239.755600   
287  223.715424  248.703125  193.032043  272.345978  293.155457  239.686966   
288  223.651642  248.626663  192.970657  272.272736  293.082214  239.618332   

          6           7           8           9    

## Find Face Area from Face Velocity and Face Flow

In [4]:
face_area_xr_results = face_flow_xr / face_velocity_xr
face_velocity_df = pd.DataFrame(face_velocity_xr.values, columns = range(nfaces))
face_flow_df = pd.DataFrame(face_flow_xr.values, columns = range(nfaces))
face_area_df_results = pd.DataFrame(face_area_xr_results.values, columns = range(nfaces))

print("Velocity")
print(face_velocity_df)
print("Flow")
print(face_flow_df)
print("Area from Results")
print(face_area_df_results)

Velocity
        0         1         2         3         4         5         6      \
0    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4    0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
..        ...       ...       ...       ...       ...       ...       ...   
284 -0.028426  0.522276  0.031250 -0.020742 -0.019684 -0.024062  0.497989   
285 -0.028297  0.519774  0.031103 -0.020435 -0.019377 -0.023995  0.495469   
286 -0.028172  0.517255  0.030958 -0.020126 -0.019063 -0.023927  0.492825   
287 -0.028045  0.514723  0.030811 -0.019819 -0.018751 -0.023858  0.490166   
288 -0.027921  0.512172  0.030664 -0.019510 -0.018435 -0.023784  0.487389   

        7         8         9      ...  11154  11155  11156     11

## Difference Between Face Area Methods

The face area method using flow and velocity is not reliable for faces at the edge of the domain. Please see results below for face 1 and 6 which were verified to be edge cells in RAS Mapper. A more rigorous analysis for all cells could be developed. Also, by inspecting the velocity data above it, the NaN appear because of dividing flow by a zero velocity.

In [5]:
face_area_Check = face_area_df_results - face_areas_df

print(face_area_Check)

        0           1         2         3         4         5           6      \
0         NaN         NaN       NaN       NaN       NaN       NaN         NaN   
1         NaN         NaN       NaN       NaN       NaN       NaN         NaN   
2         NaN         NaN       NaN       NaN       NaN       NaN         NaN   
3         NaN         NaN       NaN       NaN       NaN       NaN         NaN   
4         NaN         NaN       NaN       NaN       NaN       NaN         NaN   
..        ...         ...       ...       ...       ...       ...         ...   
284  0.001221 -237.449946  0.000824  0.001312 -0.001373 -0.001236 -238.710306   
285 -0.000885 -237.489204 -0.001129 -0.001129 -0.000763 -0.000687 -238.745263   
286 -0.000610 -237.525681 -0.000885 -0.000824 -0.000488 -0.000412 -238.781754   
287 -0.000732 -237.562140 -0.000961 -0.000946 -0.000641 -0.000504 -238.818225   
288 -0.001129 -237.595821  0.001129 -0.001373 -0.001068 -0.000931 -238.858980   

        7         8        